In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

from glob import glob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [4]:
data_txt = open('./data_split/5fold_0802.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [5]:
print(tr_data_dir['valid0'][0])

/dataset_full/Untorn/Untorn (545).png


In [39]:
selection = 4

In [40]:
add_dir = '/hdd3/dhj_container/DB'
val_data = np.zeros((len(tr_data_dir['valid%d'%selection]), total_pix), dtype=np.float32)
val_label = np.zeros((len(tr_data_dir['valid%d'%selection]), 1), dtype=np.int32)

In [41]:
for i in range(len(val_data)):
    img = cv2.imread(add_dir+tr_data_dir['valid%d'%selection][i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    val_data[i,:] = img2.flatten()
    if 'Untorn' in (add_dir+tr_data_dir['valid%d'%selection][i]):
        val_label[i] = 0
    else: val_label[i] = 1

In [43]:
f_dir1 = '/hdd3/dhj_container/ICROS_vgg/kfold0803_%d/model.ckpt-3420.meta' % selection
f_dir2 = '/hdd3/dhj_container/ICROS_vgg/kfold0803_%d/' % selection

#f_dir1 = '/models/ICROS_re/Train_3/model.ckpt-2280.meta'
#f_dir2 = '/models/ICROS_re/Train_3/'

#ev_in = test_data
#ev_in_label = test_label

ev_in = val_data
ev_in_label = val_label

saver = tf.train.import_meta_graph(f_dir1)
batch = 64
ev_steps = int(ev_in_label.shape[0]/batch)
ev_acc = np.zeros(ev_in_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob1 = graph.get_tensor_by_name("dropout6/dropout/keep_prob:0")
    keep_prob2 = graph.get_tensor_by_name("dropout7/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_in[batch*i:batch*(i+1)], keep_prob1:1.0, keep_prob2:1.0}
            label = ev_in_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_in[batch*i:], keep_prob1:1.0, keep_prob2:1.0}
            temp = ev_in_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        #print(result_pr)
        #print(result_cl)
        #print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        #print(acc)
        #print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /hdd3/dhj_container/ICROS_vgg/kfold0803_4/model.ckpt-3420


In [44]:
print(ev_acc[-1])
print(np.mean(ev_acc))
fails = np.where(ev_acc == 0)
print(len(fails[0]))

1.0
0.9923497267759562
7


In [45]:
tests = np.array([98.79, 99.12, 98.03, 98.46, 99.23])

In [46]:
print(np.mean(tests), np.std(tests))

98.72600000000001 0.4401181659509205


In [47]:
print(fails)

(array([204, 213, 215, 219, 276, 320, 614]),)
